In [ ]:
import os
import shutil

base_dir = "/content/tiny-imagenet-200"
val_dir = os.path.join(base_dir, "val")
images_dir = os.path.join(val_dir, "images")
ann_file = os.path.join(val_dir, "val_annotations.txt")

# Read annotations
with open(ann_file) as f:
    annotations = [line.strip().split('\t') for line in f]

# Create class folders and move images
for img, cls, *_ in annotations:
    cls_dir = os.path.join(val_dir, cls)
    os.makedirs(cls_dir, exist_ok=True)
    shutil.move(
        os.path.join(images_dir, img),
        os.path.join(cls_dir, img)
    )

os.rmdir(images_dir)

In [ ]:
!pip install -q tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 w

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfla
import tensorflow.keras.models as tfm
import tensorflow.keras.optimizers as tfo
import tensorflow.keras.losses as tflo
import matplotlib.pyplot as plt
from official.vision.ops import augment
import numpy as np

In [ ]:
with open("tiny-imagenet-200/wnids.txt") as f:
    wnids = [line.strip() for line in f]

train_ds = tf.keras.utils.image_dataset_from_directory(
    "tiny-imagenet-200/train",
    labels="inferred",
    label_mode="categorical",
    class_names=wnids,
    image_size=(256, 256),
    batch_size=None,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "tiny-imagenet-200/val",
    labels="inferred",
    label_mode="categorical",
    class_names=wnids,
    image_size=(224, 224),
    batch_size=128,
)

Found 100000 files belonging to 200 classes.
Found 10000 files belonging to 200 classes.


In [ ]:
def crop_image(image, label):
  # image shape: [h, w, c]
  # label shape: [num_class,]
  image = tf.image.random_crop(image, (224, 224, 3))
  image = tf.image.random_flip_left_right(image)
  return image, label

In [ ]:
train_ds = train_ds.map(crop_image, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def apply_randaugment(image, label):
  # image shape: [h, w, c]
  # label shape: [num_class,]
  augmenter = augment.RandAugment(num_layers=2, magnitude=9)
  return augmenter.distort(image), label

In [ ]:
train_ds = train_ds.map(apply_randaugment, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def normalise_image(image, label):
  # image shape: [h, w, c]
  # label shape: [num_class,]
  mean = tf.constant([0.485, 0.456, 0.406])
  std = tf.constant([0.229, 0.224, 0.225])

  image = (image / 255.0 - mean) / std

  return image, label

In [ ]:
train_ds = train_ds.map(normalise_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(normalise_image, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train_ds = train_ds.batch(128)
combined_ds = train_ds

In [ ]:
def mixup(images, labels):
  # images shape: [batchsize, h, w, c]
  # labels shape: [batchsize, num_class]
  batch_size = tf.shape(images)[0]

  gamma_1 = tf.random.gamma(shape=[batch_size, 1], alpha=0.2)
  gamma_2 = tf.random.gamma(shape=[batch_size, 1], alpha=0.2)
  lam = gamma_1 / (gamma_1 + gamma_2)

  indices = tf.random.shuffle(tf.range(batch_size))
  shuffled_images = tf.gather(images, indices)
  shuffled_labels = tf.gather(labels, indices)

  images_lam = tf.reshape(lam, [-1, 1, 1, 1])
  labels_lam = lam


  images = images_lam * images + (1 - images_lam) * shuffled_images
  labels = labels_lam * labels + (1 - labels_lam) * shuffled_labels

  return images, labels

In [ ]:
mixup_ds = train_ds.map(mixup, num_parallel_calls=tf.data.AUTOTUNE)
combined_ds = combined_ds.concatenate(mixup_ds)

In [ ]:
def cutmix(images, labels):
  # images shape: [batchsize, h, w, c]
  # labels shape: [batchsize, num_class]
  batch_size = tf.shape(images)[0]
  img_height = tf.shape(images)[1]
  img_width = tf.shape(images)[2]

  gamma_1 = tf.random.gamma(shape=[batch_size, 1], alpha=0.2)
  gamma_2 = tf.random.gamma(shape=[batch_size, 1], alpha=0.2)
  # lam is the cut percentage with shape: [batch_size, 1]
  lam = gamma_1 / (gamma_1 + gamma_2)

  # we find the cut image height and width all with shape [batch_size, 1]
  cut_height = tf.cast(tf.cast(img_height, tf.float32) * tf.sqrt(lam), tf.int32)
  cut_width = tf.cast(tf.cast(img_width, tf.float32) * tf.sqrt(lam), tf.int32)

  min_height = tf.cast(cut_height // 2, tf.int32)
  max_height = tf.cast(img_height - 1 - cut_height // 2, tf.int32)
  cut_centre_x = tf.random.uniform(shape=(batch_size, 1), minval=0, maxval=1,
                                   dtype=tf.float32)
  cut_centre_x = cut_centre_x * tf.cast(max_height - min_height, tf.float32) + tf.cast(min_height, tf.float32)

  min_width = tf.cast(cut_width // 2, tf.int32)
  max_width = tf.cast(img_width - 1 - cut_width // 2, tf.int32)
  cut_centre_y = tf.random.uniform(shape=(batch_size, 1), minval=0, maxval=1,
                                   dtype=tf.float32)
  cut_centre_y = cut_centre_y * tf.cast(max_width - min_width, tf.float32) + tf.cast(min_width, tf.float32)

  # find four conors for rectangles all with shape: [batch_size, 1]
  x1 = tf.cast(tf.cast(cut_centre_x, tf.int32) - cut_height // 2, tf.int32)
  x2 = tf.cast(tf.cast(cut_centre_x, tf.int32) + cut_height // 2, tf.int32)
  y1 = tf.cast(tf.cast(cut_centre_y, tf.int32) - cut_width // 2, tf.int32)
  y2 = tf.cast(tf.cast(cut_centre_y, tf.int32) + cut_width // 2, tf.int32)

  x_indices = tf.range(img_height)
  y_indices = tf.range(img_width)
  y_grid, x_grid = tf.meshgrid(y_indices, x_indices)
  x_grid = tf.reshape(x_grid, [1, img_height, img_width])
  y_grid = tf.reshape(y_grid, [1, img_height, img_width])

  x1 = tf.reshape(x1, [batch_size, 1, 1])
  x2 = tf.reshape(x2, [batch_size, 1, 1])
  y1 = tf.reshape(y1, [batch_size, 1, 1])
  y2 = tf.reshape(y2, [batch_size, 1, 1])

  # mask matrix with shape : [batch_size, h, w]
  mask = tf.logical_and(tf.logical_and(x1 <= x_grid, x_grid <= x2), tf.logical_and(
      y1 <= y_grid, y_grid <= y2))
  mask = tf.cast(mask, dtype=tf.int32)

  indices = tf.random.shuffle(tf.range(batch_size))
  shuffled_images = tf.gather(images, indices)
  shuffled_labels = tf.gather(labels, indices)

  mask = tf.reshape(mask, [batch_size, img_height, img_width, 1])

  images = images * tf.cast(1 - mask, dtype=tf.float32) + shuffled_images * tf.cast(mask, dtype=tf.float32)
  labels = labels * (1.0 - lam) + shuffled_labels * lam

  return images, labels

In [ ]:
cutmix_ds = train_ds.map(cutmix, num_parallel_calls=tf.data.AUTOTUNE)
combined_ds = combined_ds.concatenate(cutmix_ds)

In [ ]:
def erase(images, labels):
  # images shape: [batch_size, h, w, c]
  # labels shape: [batch_size, num_class]
  batch_size = tf.shape(images)[0]
  img_height = tf.shape(images)[1]
  img_width = tf.shape(images)[2]

  # lam is the cut percentage with shape: [batch_size, 1]
  lam = tf.random.uniform(shape=(batch_size, 1), minval=0.2, maxval=0.5, dtype=tf.float32)

  # we find the cut image height and width all with shape [batch_size, 1]
  erase_height = tf.cast(tf.cast(img_height, tf.float32) * tf.sqrt(lam), tf.int32)
  erase_width = tf.cast(tf.cast(img_width, tf.float32) * tf.sqrt(lam), tf.int32)

  min_height = tf.cast(erase_height // 2, tf.int32)
  max_height = tf.cast(img_height - 1 - erase_height // 2, tf.int32)
  erase_centre_x = tf.random.uniform(shape=(batch_size, 1), minval=0, maxval=1,
                                   dtype=tf.float32)
  erase_centre_x = erase_centre_x * tf.cast(max_height - min_height, tf.float32) + tf.cast(min_height, tf.float32)

  min_width = tf.cast(erase_width // 2, tf.int32)
  max_width = tf.cast(img_width - 1 - erase_width // 2, tf.int32)
  erase_centre_y = tf.random.uniform(shape=(batch_size, 1), minval=0, maxval=1,
                                   dtype=tf.float32)
  erase_centre_y = erase_centre_y * tf.cast(max_width - min_width, tf.float32) + tf.cast(min_width, tf.float32)

  # find four conors for rectangles all with shape: [batch_size, 1]
  x1 = tf.cast(tf.cast(erase_centre_x, tf.int32) - erase_height // 2, tf.int32)
  x2 = tf.cast(tf.cast(erase_centre_x, tf.int32) + erase_height // 2, tf.int32)
  y1 = tf.cast(tf.cast(erase_centre_y, tf.int32) - erase_width // 2, tf.int32)
  y2 = tf.cast(tf.cast(erase_centre_y, tf.int32) + erase_width // 2, tf.int32)

  x_indices = tf.range(img_height)
  y_indices = tf.range(img_width)
  y_grid, x_grid = tf.meshgrid(y_indices, x_indices)
  x_grid = tf.reshape(x_grid, [1, img_height, img_width])
  y_grid = tf.reshape(y_grid, [1, img_height, img_width])

  x1 = tf.reshape(x1, [batch_size, 1, 1])
  x2 = tf.reshape(x2, [batch_size, 1, 1])
  y1 = tf.reshape(y1, [batch_size, 1, 1])
  y2 = tf.reshape(y2, [batch_size, 1, 1])

  # mask matrix with shape : [batch_size, h, w]
  mask = tf.logical_and(tf.logical_and(x1 <= x_grid, x_grid <= x2), tf.logical_and(
      y1 <= y_grid, y_grid <= y2))
  mask = tf.cast(mask, dtype=tf.int32)

  mask = tf.reshape(mask, [batch_size, img_height, img_width, 1])

  images = images * tf.cast(1 - mask, dtype=tf.float32)

  return images, labels

In [ ]:
erase_ds = train_ds.map(erase, num_parallel_calls=tf.data.AUTOTUNE)
combined_ds = combined_ds.concatenate(erase_ds)

In [ ]:
def label_smoothing(labels, epsilon=0.1):
  num_class = tf.cast(tf.shape(labels)[1], tf.float32)
  return labels * (1.0 - epsilon) + epsilon / num_class

In [ ]:
combined_ds = combined_ds.map(lambda images, labels: (images, label_smoothing(labels)), num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
combined_ds = combined_ds.shuffle(buffer_size=100)

In [ ]:
class block(tfla.Layer):
  def __init__(self, C):
    super().__init__()
    self.C = C

    self.dwconv = tfla.DepthwiseConv2D(
        kernel_size=7,
        strides=1,
        padding="same",
        depth_multiplier=1,
        use_bias=True
    )

    self.layernor = tfla.LayerNormalization(epsilon=1e-6)

    self.pw1 = tfla.Dense(C * 4, activation="gelu", use_bias=True)
    self.pw2 = tfla.Dense(C, use_bias=True)

    self.gamma = self.add_weight(
        name="gamma",
        shape=[1, 1, 1, C],
        initializer=tf.initializers.Constant(1e-6),
        trainable=True
    )

  def dropLayer(self, x, drop_rate, training):
    # x shape:[B, H, W, C]
    B = tf.shape(x)[0]

    if not training or drop_rate == 0:
      return x

    keep_rate = 1 - drop_rate

    mask = tf.random.uniform(shape=(B, 1, 1, 1), minval=0.0, maxval=1.0, dtype=tf.float32)
    mask = tf.floor(mask + keep_rate)

    x = x * tf.cast(mask, tf.float32) / keep_rate

    return x

  def call(self, x, training=False):
    # x shape:[B, H, W, C]
    shortcut = x
    x = self.dwconv(x)
    x = self.layernor(x)
    x = self.pw1(x)
    x = self.pw2(x)

    x = self.gamma * x

    x = self.dropLayer(x, 0.1, training)

    return x + shortcut

In [ ]:
class down_sample(tfla.Layer):
  def __init__(self, out_dim):
    super().__init__()
    self.layernor = tfla.LayerNormalization(epsilon=1e-6)
    self.conv = tfla.Conv2D(filters=out_dim, kernel_size=2, strides=2, padding="valid")

  def call(self, x):
    x = self.layernor(x)
    x = self.conv(x)
    return x

In [ ]:
inputs = tfla.Input(shape=(224, 224, 3))

x = tfla.Conv2D(filters=96, kernel_size=4, strides=4, padding="valid", use_bias=True)(inputs)
x = tfla.LayerNormalization(epsilon=1e-6)(x)

for _ in range(3):
  x = block(C=96)(x)

x = down_sample(out_dim=192)(x)

for _ in range(3):
  x = block(C=192)(x)

x = down_sample(out_dim=384)(x)

for _ in range(9):
  x = block(C=384)(x)

x = down_sample(out_dim=768)(x)

for _ in range(3):
  x = block(C=768)(x)

x = tfla.GlobalAveragePooling2D()(x)
x = tfla.LayerNormalization(epsilon=1e-6)(x)
x = tfla.Dense(units=200, activation="softmax")(x)

model = tfm.Model(inputs=inputs, outputs=x)

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 56, 56, 96)     │         4,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 56, 56, 96)     │           192 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block (block)                   │ (None, 56, 56, 96)     │        79,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1 (block)                 │ (None, 56, 56, 96)     │        79,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_2 (block)                 │ (None, 56, 56, 96)     │        79,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_sample (down_sample)       │ (None, 28, 28, 192)    │        74,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_3 (block)                 │ (None, 28, 28, 192)    │       306,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_4 (block)                 │ (None, 28, 28, 192)    │       306,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_5 (block)                 │ (None, 28, 28, 192)    │       306,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_sample_1 (down_sample)     │ (None, 14, 14, 384)    │       295,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_6 (block)                 │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_7 (block)                 │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_8 (block)                 │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_9 (block)                 │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_10 (block)                │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_11 (block)                │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_12 (block)                │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_13 (block)                │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_14 (block)                │ (None, 14, 14, 384)    │     1,201,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ down_sample_2 (down_sample)     │ (None, 7, 7, 768)      │     1,181,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_15 (block)                │ (None, 7, 7, 768)      │     4,763,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_16 (block)                │ (None, 7, 7, 768)      │     4,763,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_17 (block)                │ (None, 7, 7, 768)      │     4,763,136 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 27,973,928 (106.71 MB)

 Trainable params: 27,973,928 (106.71 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
steps_per_epoch = 3128
epochs = 40
total_steps = steps_per_epoch * epochs

lr = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=5e-4,
    decay_steps=total_steps,
    alpha=1e-2
)

opt = tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=5e-2)

In [ ]:
optimizer = tfo.AdamW(learning_rate=0.001, weight_decay=0.05, beta_1=0.9, beta_2=0.999)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

history = model.fit(
    combined_ds,
    epochs=40,
)

Epoch 1/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1429s 418ms/step - accuracy: 0.0227 - loss: 5.2228
Epoch 2/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1264s 399ms/step - accuracy: 0.1093 - loss: 4.5220
Epoch 3/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1265s 400ms/step - accuracy: 0.1822 - loss: 4.1156
Epoch 4/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1263s 399ms/step - accuracy: 0.2480 - loss: 3.8023
Epoch 5/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1263s 399ms/step - accuracy: 0.3105 - loss: 3.5224
Epoch 6/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1265s 399ms/step - accuracy: 0.3713 - loss: 3.2671
Epoch 7/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1264s 399ms/step - accuracy: 0.4245 - loss: 3.0441
Epoch 8/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1265s 400ms/step - accuracy: 0.4747 - loss: 2.8538
Epoch 9/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1264s 399ms/step - accuracy: 0.5150 - loss: 2.7031
Epoch 10/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1263s 399ms/step - accuracy: 0.5516 - loss: 2.5669
Epoch 11/40
3128/3128 ━━━━━━━━━━━━━━━━━━━━ 1263s 399ms/step - accurac

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='lower right')
plt.show()

In [ ]:
model.evaluate(test_ds)

79/79 ━━━━━━━━━━━━━━━━━━━━ 28s 291ms/step - accuracy: 0.6017 - loss: 1.7162


[1.697739601135254, 0.6047999858856201]